In [1]:
import laspy
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import box, LineString

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Loading

In [47]:
las = laspy.read('data/HT412_1743085214_3237183_1427127638184903_1427127806163693.las')
las.header

<LasHeader(1.2, <PointFormat(0, 0 bytes of extra dims)>)>

In [48]:
las.header.parse_crs()

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

# Converting to EPSG:4326

In [49]:
X_corrected = las.X * las.header.scales[0] + las.header.offsets[0]
Y_corrected = las.Y * las.header.scales[1] + las.header.offsets[1]
Z_corrected = las.Z * las.header.scales[2] + las.header.offsets[2]

In [50]:
X_corrected.min(), X_corrected.max(), Y_corrected.min(), Y_corrected.max(), Z_corrected.min(), Z_corrected.max()

(np.float64(11.605219216536979),
 np.float64(11.652211816536978),
 np.float64(48.20662813364922),
 np.float64(48.22857350864922),
 np.float64(520.7259527041539),
 np.float64(571.7734527041539))

In [51]:
data = np.stack([las.X, las.Y, las.Z, X_corrected, Y_corrected, Z_corrected, las.intensity, las.scan_angle_rank, las.user_data, las.point_source_id], axis=0).T
data = pd.DataFrame(data)
data.columns = ['X',
                'Y',
                'Z',
                'X_corrected',
                'Y_corrected',
                'Z_corrected',
                'intensity',
                'scan_angle_rank',
                'user_data',
                'point_source_id']

# Mapping

In [53]:
x_min, x_max, y_min, y_max = X_corrected.min() , X_corrected.max(), Y_corrected.min() , Y_corrected.max()

center_lat = (y_min + y_max) / 2
center_lon = (x_min + x_max) / 2

N = 10000000

chunk_indices = np.arange(len(data)) // N
reduced = data.groupby(chunk_indices)[['X_corrected', 'Y_corrected']].median().reset_index(drop=True)

coords = list(zip(reduced['X_corrected'], reduced['Y_corrected']))

line = LineString(coords)
gdf = gpd.GeoDataFrame({'geometry': [line]}, crs='EPSG:4326')

m = folium.Map(location=[center_lat, center_lon], zoom_start=14, tiles='OpenStreetMap')

folium.GeoJson(gdf).add_to(m)

m.save("bounding_box_map.html")
m